I'm trying to ensemble these 7 notebooks, along with using a species classifier to down-prioritize any results that are the wrong species, since the species classifier is more accurate than the individual results.

The original ensemble notebook : https://www.kaggle.com/code/yamsam/simple-ensemble-of-public-best-kernels  

I'm using a species classifier on the test dataset from this FastAI kernel https://www.kaggle.com/code/kwentar/species-classification/notebook.  This idea didn't help.  It seems that between 7 classifiers, it seems species deliniation is actually pretty good.  A better approach might have been to use the classifier to build seperate models for each species.  Or maybe just Dolphin/Whale/Beluga Whale since those all had quite different distinguishing features.

---

Ensembelling the following kernels:

* https://www.kaggle.com/code/nghiahoangtrung/swin-tranform-submission  (0.764 SWIN Transform, backfin TFrecords)

*  https://www.kaggle.com/aikhmelnytskyy/happywhale-arcface-baseline-eff7-tpu-768-inference  (0.729 ArcFace Eff7 768 5 Folds)

* https://www.kaggle.com/code/clemchris/pytorch-backfin-convnext-arcface  (0.725 Best82. Output CSV dataset Only)

* https://www.kaggle.com/nghiahoangtrung/0-720-eff-b5-640-rotate (0.72 Eff B5 With Rotation, this one is already using a cropped Backfin dataset)

* https://www.kaggle.com/aikhmelnytskyy/happywhale-effnet-b7-fork-with-detic-training  (0.699 ArcFace Effnet B7 + Detic Cropping)

* https://www.kaggle.com/code/ollypowell/fastai-baseline-model (0.682 FastAI ArcFace, Backfin TFrecords)

* https://www.kaggle.com/code/librauee/arcfaceeffb6inferbaseline  (0.655 Effnet B6, I should try and improve on this with SOD, object detect, horizontal flip + rotate, more folds)


The dataset for the species classification provided as a .csv seperately here: https://www.kaggle.com/datasets/kwentar/happywhale-test-species


In [1]:
# List the submission paths and the weightings, using public leaderboard scores

import csv
import pandas as pd 

sub_files = [    '../input/swin-tranform-submission/submission.csv',  # LB 0.764 I added this, it made a big improvement
                 '../input/happywhale-arcface-baseline-eff7-tpu-768-inference/submission.csv',  # LB 0.729 Not much I can do to imrove this one in time available
                 '../input/best82/submission.csv',  # LB 0.725  Not much I can do to imrove this one in time available
                 '../input/0-720-eff-b5-640-rotate/submission.csv', # LB 0.72
                 '../input/happywhale-effnet-b7-fork-with-detic-crop/submission.csv', #  0.699 
                 '../input/fastai-baseline-model/submission.csv',  #0.682
                 '../input/arcfaceeffb6inferbaseline/submission.csv' # 0.655  Should try with improved dataset & augmentation on colab              
]

# Weights of the individual subs    I'm using this fomula, it is only fitted to the leaderboard in the sense that it uses the public scores, as biases
# Another version of this notebook looked like somebody had hand tweaked the biases to fit the public leaderboard, I doubt that would generalise well.

N_wight=0.5
Base_wight=1.5
SPECIES_PENALTY = 1   # The penalty if the id predictions do not match the expected species from the classifier

l1 = [0.764, 0.729, 0.725, 0.72, 0.699, 0.682, 0.655]    # Relative weightings of the datasets
l2 = [Base_wight+N_wight*i for i in range(len(l1))]    # Formula
sub_weight = list(map(lambda x,y: x**y ,l1,l2))
print('The submission weightings: ', sub_weight)

The submission weightings:  [0.6677901945970756, 0.5314409999999999, 0.44755356042950317, 0.37324799999999997, 0.2855420775971462, 0.21634033537600006, 0.1489655519011116]


In [2]:
#Read the species classification file and make a image file - species dictionary

df = pd.read_csv (r'../input/happywhale-test-species/Happywhale_test_species.csv')
print(df.head())

species_dict = dict(zip(df.image, df.species))

print('\n Dictionary key-value tuples: ', list(species_dict.items())[:5])

                image  label         species
0  000110707af0ba.jpg      8      gray_whale
1  0006287ec424cb.jpg      1  humpback_whale
2  000809ecb2ccad.jpg      4          beluga
3  00098d1376dab2.jpg      1  humpback_whale
4  000b8d89c738bd.jpg     13   dusky_dolphin

 Dictionary key-value tuples:  [('000110707af0ba.jpg', 'gray_whale'), ('0006287ec424cb.jpg', 'humpback_whale'), ('000809ecb2ccad.jpg', 'beluga'), ('00098d1376dab2.jpg', 'humpback_whale'), ('000b8d89c738bd.jpg', 'dusky_dolphin')]


In [3]:
#Load the CSV training file and correct for some misspelt entries

train_df = pd.read_csv(r'../input/happy-whale-and-dolphin/train.csv')
train_df.species.replace({"globis": "short_finned_pilot_whale",
                          "pilot_whale": "short_finned_pilot_whale",
                          "kiler_whale": "killer_whale",
                          "bottlenose_dolpin": "bottlenose_dolphin"}, inplace=True)
print(train_df.head())

sorted(train_df.species.unique()), len(train_df.species.unique())

                image             species individual_id
0  00021adfb725ed.jpg  melon_headed_whale  cadddb1636b9
1  000562241d384d.jpg      humpback_whale  1a71fbb72250
2  0007c33415ce37.jpg  false_killer_whale  60008f293a2b
3  0007d9bca26a99.jpg  bottlenose_dolphin  4b00fe572063
4  00087baf5cef7a.jpg      humpback_whale  8e5253662392


(['beluga',
  'blue_whale',
  'bottlenose_dolphin',
  'brydes_whale',
  'commersons_dolphin',
  'common_dolphin',
  'cuviers_beaked_whale',
  'dusky_dolphin',
  'false_killer_whale',
  'fin_whale',
  'frasiers_dolphin',
  'gray_whale',
  'humpback_whale',
  'killer_whale',
  'long_finned_pilot_whale',
  'melon_headed_whale',
  'minke_whale',
  'pantropic_spotted_dolphin',
  'pygmy_killer_whale',
  'rough_toothed_dolphin',
  'sei_whale',
  'short_finned_pilot_whale',
  'southern_right_whale',
  'spinner_dolphin',
  'spotted_dolphin',
  'white_sided_dolphin'],
 26)

In [4]:
#Use the training csv file to create an individual-species dictionary

print ('There are ', len(train_df.individual_id.unique()), 'unique individuals')

dict1 =  dict(zip(train_df.individual_id, train_df.species))     # Will contain duplicates  
temp = {val: key for val, key in dict1.items()}
id_species = {val: key for val, key in temp.items()}

print ('There are ', len(id_species.items()), 'entries in id_species dictionary')

There are  15587 unique individuals
There are  15587 entries in id_species dictionary


In [5]:
#Read and order the CSV output files from each submission

Hlabel = 'image' 
Htarget = 'predictions'
npt = 6
place_weights = {}
for i in range(npt):
    place_weights[i] = (1 / (i + 1))   # Makes this: {0: 1.0, 1: 0.5, 2: 0.3333333333333333, 3: 0.25, 4: 0.2, 5: 0.16666666666666666}

print(place_weights)

lg = len(sub_files)    # Number of files to ensemble
sub = [None]*lg        # Empty String with that number of values = None
for i, file in enumerate( sub_files ):          #Loop through the filepath strings, incrementing i from 0 to one less than number of files
    print("Reading {}: w={} - {}". format(i, sub_weight[i], file))    
    reader = csv.DictReader(open(file,"r"))               #Open a file with DictReader
    sub[i] = sorted(reader, key=lambda d: str(d[Hlabel])) #sorted dictionary by the Hlabel key (makes sure 'image' first, then 'predictions')

print(len(sub)) #=6 
print(len(sub[0])) #= 27956
print(sub[0][:3]) # sub is a list of ordered dictionaries  
#OrderedDict([('image', '000110707af0ba.jpg'), ('predictions', 'fbe2b15b5481 new_individual 54a19a45715c a68efd897f36 74062c624dea')])   x 27956, x 6


{0: 1.0, 1: 0.5, 2: 0.3333333333333333, 3: 0.25, 4: 0.2, 5: 0.16666666666666666}
Reading 0: w=0.6677901945970756 - ../input/swin-tranform-submission/submission.csv
Reading 1: w=0.5314409999999999 - ../input/happywhale-arcface-baseline-eff7-tpu-768-inference/submission.csv
Reading 2: w=0.44755356042950317 - ../input/best82/submission.csv
Reading 3: w=0.37324799999999997 - ../input/0-720-eff-b5-640-rotate/submission.csv
Reading 4: w=0.2855420775971462 - ../input/happywhale-effnet-b7-fork-with-detic-crop/submission.csv
Reading 5: w=0.21634033537600006 - ../input/fastai-baseline-model/submission.csv
Reading 6: w=0.1489655519011116 - ../input/arcfaceeffb6inferbaseline/submission.csv
7
27956
[OrderedDict([('image', '000110707af0ba.jpg'), ('predictions', 'fbe2b15b5481 new_individual 54a19a45715c a68efd897f36 74062c624dea')]), OrderedDict([('image', '0006287ec424cb.jpg'), ('predictions', '1424c7fec826 new_individual c3b7d902e73c 08e7aae3e88e 98995caaa0d3')]), OrderedDict([('image', '000809ecb2

### Write the output file

In [6]:
out = open("submission.csv", "w", newline='')
writer = csv.writer(out)
writer.writerow([Hlabel,Htarget])     #This is writing the header row  "image", "predictions"

wrong_species = 0 
correct_species = 0
new_individual = 0
#changed = 0

for p, row in enumerate(sub[0]):     #iterate the rows of each image
    target_weight = {}
    for s in range(lg):              #iterate results of each kernel in sub   
        row1 = sub[s][p]             #row1 is from the sth kernel, and the pth ordered dic.  
        #example: [('image', '000110707af0ba.jpg'), ('predictions', 'fbe2b15b5481 new_individual 54a19a45715c a68efd897f36 74062c624dea')])
        image_file = row1[Hlabel]
        for ind, trgt in enumerate(row1[Htarget].split(' ')):  #gets each prediction   ind is the position, trgt is the id
            # a dictionary with each target, adding the sum from previous kernels estimates + place_weight*sub_weight 
            #target_weight[trgt] = target_weight.get(trgt,0) + (place_weights[ind]*sub_weight[s]) 
            #So I'm modifying this to penalise predictions that are not the expected species
            image_species = species_dict[image_file]   # produces a string corresponding to the species
            if trgt == 'new_individual':
                species_weight = 1
                new_individual += 1        
            else: 
                if id_species[trgt] == image_species:     #Both Strings.
                    species_weight = 1   
                    correct_species += 1
                else: 
                    species_weight = SPECIES_PENALTY  # Wrong species penalty
                    wrong_species +=1
            
            target_weight[trgt] = target_weight.get(trgt,0) + (place_weights[ind]*sub_weight[s]*species_weight) 
    
    #if max(list(target_weight.values())) < 0.01:  # The case when they are all 'wrong species'  It would be better to guess new individual
        #tops_trgt = ['new_individual'] + sorted(target_weight, key=target_weight.get, reverse=True)[:npt-1]
        #changed +=1
    #else:            
    tops_trgt = sorted(target_weight, key=target_weight.get, reverse=True)[:npt]   # a list of targets sorted by weight  1 extra so I can add a new individual
    
    writer.writerow([row1[Hlabel], " ".join(tops_trgt)])
print(target_weight)
out.close()

#print(wrong_species, correct_species, new_individual, changed)  # This didn't help.  The kernels seem to be doing a pretty good job of getting the species right!

{'32a8f92d7809': 1.7049320904025789, 'new_individual': 1.335440359950418, '64bb5df01cdf': 0.506507331018825, '556d610a3896': 0.2732357486492689, 'f187914208a4': 0.18764312276341513, 'ad4557cf336c': 0.6749399645246199, '4587a8786179': 0.41840232759714624, 'a7a5696645cb': 0.14918452014316771, 'f7ce926d1f9a': 0.11188839010737579, '741f7de87a7b': 0.08951071208590064, '783f357cf2f9': 0.2733815519011116, '18dbd44823c7': 0.09331199999999999, 'ad981a124bbd': 0.0746496, '44a2f97c19c3': 0.09518069253238207, '7ef48fb30101': 0.04326806707520001, '5b7ef1ff1875': 0.0372413879752779, 'e59ada492aeb': 0.029793110380222323}
